# PS 88 Week 11 Class Notebook

In [ ]:
import numpy as np
import seaborn as sns
import statsmodels.formula.api as smf
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

Often times the relationship between variables is not linear, which is assumed by the basic regressions we have run so far. Here is an example with simulated data:

In [ ]:
x = np.random.normal(1,1,size=100)
y  = 2*x - 3*x**2 + np.random.normal(0,1, size=100)
dat = pd.DataFrame(data={"x":x, "y":y})
dat

If we draw a "best fit line" through this x and y it is clear that we are not fitting the data well: the relationship is "curved".

In [ ]:
sns.regplot(...)

We can still run a bivariate regression which tries to predict y given x in a linear fashion with `smf.ols`:

In [ ]:
smf.ols(...).fit().summary()

Here we get a reasonably strong negative slope (which is statistically significant), so without looking at the graph we might just declare vitory and move on.

However, we can describe this relatinship much better by adding a *quadratic term*, or effectively fitting a "best fit parabola" through the data. That is, rather than finding an $a$ and $b_1$ to minimize the error terms in a bivariate linear regression:
$$
Y_i = a + b_1 X_i + e_i
$$
we can find an $a$, $b_1$, and $b_2$ to minimize the error terms which result from the following equation:
$$
Y_i = a + b_1 X_i + b_2 X_i^2 + e_i
$$

An easy way to do this visually is to use the `sns.regplot` function and add an `order=2` argument, which essentially says "fit a second-order polynomial", which is the quadratic function above.

Clearly the error terms are much smaller here! To see this formally we can use `smf.ols` to find the $a$, $b_1$, and $b_2$ which best fit the data. To add the quadratic term we add a `+ I(x**2)` term in our regression formula:

We can see the fit is much better because the $R^2$ value goes way up to .97.

How can we interpret these coefficients? The slope of the relationship between $x$ and $y$ is no longer constant: this is what it means for the relationship to be nonlinear!

If our best fit cure is given by $y = a + b_1 x + b_2 x^2$, then by a standard calculus calculation, the slope *as a function of $x$ is now:
$$
b_1  + 2 b_2 x
$$

Note that if $x=0$, the slope is just $b_1$. So the coefficient on our "linear term" (1.99 in this example) tells us the slope at $x=0$. (If you look back at the regplot you should be able to see the curve is increasing at $x=0$.  

One way we can interpret the $b_2$ coefficient (-2.9 above) is how the slope changes as x increases. Since this is negative, the slope gets more negative as $x$ increases. And, as we can see in the regplot, the slope is positive when $x$ is negative/small, and then gets negative when $x$ is higher. We often call this a "inverse u-shaped curve". This general shape will arise when there is a negative coefficient on the quadratic term. Here is another example:

In [ ]:
x2 = np.random.normal(1,1,size=100)
y2  = -2*x2 + 3*x2**2 + np.random.normal(0,1, size=100)
dat2 = pd.DataFrame(data={"x":x2, "y":y2})
sns.regplot(x='x', y='y', data=dat2, order =2, ci=0)


Here the curve is "flipped", and the slope is negative for low x and then gets more positive. 

Note we now have a positive squared term. 

Combining, here is how to interpret the coefficients in regressions with a squared term:
- The linear term always tells us the slope when our independent variable is equal to zero
- If the squared term coefficient is postive, the slope increases as x increases, creating a u-shaped curve. If the squared term coefficient is negative, the slope decreases as x increases, creating a inverse-u shaped curve.

We can go beyond using squared terms, and add a "cubed term", etc. This will allow us to more flexibly fit the data. Here is a more complicated example:

In [ ]:
y2 = x + .5*x**3 -  10*np.sin(2*x) + np.random.normal(0, 1, 100)
dat['y2'] = y2
sns.regplot(x=x, y=y2, data=dat, ci=0)

Here a linear term doesn't fit very well at all. Let's try with a quadratic term by adding an `order=2` argument to `sns.regplot`:

That's a bit better, but it still doesn't fit the curve very well. We can always just up our order argument to have a more flexible curve:

That works better! 

For toy examples like this we can just add more and more polynomial terms (a higher order argument) and can fit the data pretty well. However, this comes at a cost of making the slopes from a regression much harder to interpret. We will typically focus on cases with just a quadratic term in real examples.

## Interaction terms

Now let's consider the possibility that the relationship between our independent variable and dependent variable changes as another independent variable changes. That may sound pretty abstract, so let's see an example with the data from the lab last week on ACA voting and the 2010 midterms.

In [ ]:
hcr_mid = pd.read_stata("hcr_midterm.dta")
hcr_mid

Recall we want to focus on democrats who were in competitive districts

In [ ]:
hcr_mid = hcr_mid[(hcr_mid["dem_n"] > 0) & (hcr_mid["party"]=="D")]
hcr_mid

The multivariate regression we considered before

In [ ]:
ols_mv = smf.ols('dem_n ~ hcr_yes + obama', data=hcr_mid).fit()
ols_mv.summary()

Visualizing this regression

In [ ]:
sns.scatterplot(x='obama', y='dem_n', data=hcr_mid_y, color="green")
sns.scatterplot(x='obama', y='dem_n', data=hcr_mid_n, color="orange")
xrange = np.arange(30, 100)
plt.plot(xrange, ols_mv.params[0] + xrange*ols_mv.params[2], color="orange")
plt.plot(xrange, ols_mv.params[0] + ols_mv.params[1] + xrange*ols_mv.params[2], color="green")

If you eyeball the graph, it looks like the slope of the yellow points (no votes) may be less steep than the slope of the green dots. Let's check that by running seperate regplots on the two groups.

In [ ]:
hcr_mid_y = hcr_mid[hcr_mid["hcr_yes"] == 1]
hcr_mid_n = hcr_mid[hcr_mid["hcr_yes"] == 0]

Note that the orange line gives the prediction of how well the Democratic member of congress would do in the midterm as a function of Obama vote share when voting no, and the blue line is the prediction if voting yes. So when the orange line is higher, these linear regressions would predict that the member of congress would do better voting no, while when the blue line is higher we predict the Democratic member of congress would do better when voting yes. we can see here that the lines cross around a 58% vote share for Obama. So, for those in conservative/moderate districts we predict that voting yes leads to fewer votes, while for this in more liberal districts voting yes is associated with a higher vote share.


If we step back and think about the context, this makes a lot of sense. The ACA was broadly viewed as a liberal piece of legislation (not as much as some would like though!). Districts where Obama did worse in 2008 likely have a lot of more moderate to conservative voters, who did not like his signature legislation. So, voters may have punished Democrats who voted for the bill in such districts. On the other hand, in more liberal districts where Obama did well we should expect voters wanted their representatives to vote *yes*. 

More generally, the relationship between the ACA vote and midterm performance should be different for districts with low and high Obama vote share. And that is exactly what we find!

We can do a similar analysis by running one regression with an *interaction term*. In equation form, we find an $a$, $b_1$, $b_2$, and $b_3$ which minimize the squared error terms for the following equation:
$$
Y_i = a + b_1 X_i + b_2 Z_i + b_3 X_i \times Z_i + \epsilon_i
$$
where in this example $Y_i$ is the Democrat midterm vote share, $X_i$ is the vote for the ACA, and $Y_i$ is the Obama vote share.

Once we get our coefficients (which we will do in a second), we can predict the vote share for someone who voted no by plugging in $X_i=0$ to this equation, which gives:
$$
Y_i = a + b_2 Z_i + \epsilon_i
$$
and for someone who votes yes ($X_i =1$) we get:
$$
Y_i = a + b_1 + (b_2 + b_3) Z_i + \epsilon_i
$$

In other words, we are estimating a different slope and intercept for the $X_i=0$ case (no votes) and the $X_i =1$ case (yes votes).

Let's see how this looks in the ACA example. We can include the interaction term by adding a "hcr_yes:obama" to the regression formula.,

Recall that the "hcr_yes" (or $X_i$) variable is equal to 0 or 1. When it is equal to zero, we are left with the following prediction:
$$
\text{dem_n} = 24.41 + 0.504 \times \text{Obama}
$$
for people who vote yes we get:
$$
\text{dem_n} = 24.41 - 27.6 + (0.504 + .484) \times \text{Obama}
$$
If we want to know the difference in the prediction for vote share when voting yes vs no we subtract the first from the second which gives:
$$
-27.6 + .484 \times \text{Obama}
$$
This tells us that in a hypothetical district where Obama got 0 votes, the predicted change in vote share when voting for the ACA is a decline of 27.6%. This is huge, but of course there are no districts represented by Democrats where Obama got no votes!

We To see the prediction for a district with a 50% obama vote share we can plug this into the formula above:

For such a district we estimate a small (but meaningful) penalty of losing about 3.4% of the vote. How about a "safe district" where Obama won 75% of the vote?


In such a district those who vote for the ACA are predicte to do better. We can recreate the "two regression" plot by plotting the predictions for yes and no voters separately.

In [ ]:
ols_int.params

Checking that our interactive regression model model gives the graph

In [ ]:
sns.scatterplot(x='obama', y='dem_n', data=hcr_mid_y, color="green")
sns.scatterplot(x='obama', y='dem_n', data=hcr_mid_n, color="orange")
xrange = np.arange(30, 100)
plt.plot(xrange, ols_int.params[0] + xrange*ols_int.params[2], color="orange")
plt.plot(xrange, ols_int.params[0] + ols_int.params[1]+ 
         (ols_int.params[2] + ols_int.params[3])*xrange, color="green")

More broadly, this interaction term tells us how the predicted change $Y$ as $X$ goes up by one unit changes as $Y$ goes up by one unit. A positive interaction means that as $Z$ increases the relationship between $X$ and $Y$ becomes more positive, and the reverse is true if negative.